# Faultproof Withdrawals Triage Runbook
In order to start this runbook locally please use the README located at [https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md](https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md)

General instructions to start the runbook locally:
```bash
cd op-monitorism/faultproof_withdrawals/runbooks/automated/
make start
```


In [1]:
from dotenv import load_dotenv
import os
from web3 import Web3
from lib.superchain import *
from lib.web3 import *
from pprint import pprint
from datetime import datetime, timedelta
from IPython.display import Javascript, display
import yaml


#parameters setup (there should be no need to change the one below)
abi_folder_path="abi"

env_file = ".env"


## setup 
In order to proceed make sure you set the parameters below.
The url used below are the trusted nodes that are going to be used.

In [2]:
#set ignore_certificate if you are using a local https
ignore_url_certificate=True

#set cahin you need to monitor
l1_chain_name="mainnet"
l2_chain_name="base"

#set the chain URL to use as trusted node explicitly or set in .env file
L1_GETH_URL = ""
L2_NODE_URL = ""
L2_GETH_URL = ""    

## Loading local values and superchain regsitry values
You can either set the values manually temporarly
```
L1_GETH_URL = ""
L2_NODE_URL = ""
L2_GETH_URL = ""  
```
Or you can set into .env for a more permanent use of this runbooks.
Make sure to use the correct name convention as shown below.
```
mainnet_geth_url="https://l1-geth.rpc"
<l1>_geth_url="https://l1-geth.rpc"

mainnet_op_geth_url=
<l1>_<l2>_get_url="https://op-geth.rpc"

mainnet_op_node_url="https://op-node.rpc"
<l1>_<l2>_node_url="https://op-geth.rpc"
```


If you are part of OP you have access to the internal repository with internal nodes to call. You can save [https://github.com/ethereum-optimism/internal-data/blob/main/automated-runbooks/.env](https://github.com/ethereum-optimism/internal-data/blob/main/automated-runbooks/.env) whitin the same folder.

In [ ]:
if L1_GETH_URL == "" or L2_NODE_URL == "" or L2_GETH_URL == "":
    env_file = ".env"
    if os.path.exists(env_file):
        load_dotenv(env_file)
        L1_GETH_URL = os.getenv((f"{l1_chain_name}_geth_url"))
        L2_OP_NODE_URL = os.getenv((f"{l1_chain_name}_{l2_chain_name}_node_url"))
        L2_OP_GETH_URL = os.getenv((f"{l1_chain_name}_{l2_chain_name}_geth_url"))
    else:
        print("No .env file found. Using system environment variables. Make sure to set them up. An example .env file is provided insite the automated folder .env.example")
        print("If you have access to private nodes in OP Labs you can download the .env file from https://github.com/ethereum-optimism/internal-data/blob/main/automated-runbooks/.env")
        
if L1_GETH_URL is None or L2_NODE_URL is None or L2_GETH_URL is None:
    print("Please set the environment variables for the chains you want to monitor")
    exit()
    
superchain=get_superchain_file(l1_chain_name, l2_chain_name)

eth_scan_url="https://etherscan.io"

if l1_chain_name=="sepolia":
    eth_scan_url="https://sepolia.etherscan.io"

l2_eth_scan_url=superchain["explorer"]

OptimismPortalProxy=superchain["addresses"]["OptimismPortalProxy"]

print(f"OptimismPortal2 address: {eth_scan_url}/address/{OptimismPortalProxy}#readProxyContract")

web3_utility=Web3Utility(L1_GETH_URL, L2_OP_GETH_URL,L2_OP_NODE_URL,abi_folder_path, OptimismPortalProxy, ignore_certificate=ignore_url_certificate)

# Triage alert: faultproof-withdrawal-forgery-detection-stalled
This alert monitors the number of withdrawal events that are considered normal in a chain. If the number of withdrawal events goes below a specified threshold, we trigger this alert.

We want to verify when it was last withdrawals on the chain. If there are no provenwithdrawals events on the chain in the last day, then we know the alert is a false positive and there is no issue with the monitor.

In [ ]:
result=web3_utility.find_latest_withdrawal_event()

log=result["log"]
timestamp=result["timestamp"]

print(f"Last withdrawal event:\ntimestamp: {timestamp["timestamp"]} \ntimestamp: {timestamp["formatted_timestamp"]}\n")
print(f"Last withdrawal happened: \nat block: {eth_scan_url}/block/{log["blockNumber"]} \nwith transaction hash: {eth_scan_url}/tx/0x{log["transactionHash"].hex()}")
print(f"Logs can be found at: {eth_scan_url}/address/{OptimismPortalProxy}#events using filter:0x798f9f13695f8f045aa5f80ed8efebb695f3c7fe65da381969f2f28bf3c60b97")

# Convert the Unix timestamp to a datetime object
event_time = datetime.fromtimestamp(timestamp["timestamp"])

# Check if the event timestamp is more than 24 hours ago
if event_time < (datetime.now() - timedelta(hours=24)):
    print("\nLast withdrawal event happened more than 24 hours ago")
else:
    print("\nWARNING: Last withdrawal event happened less than 24 hours ago. The monitoring may be stalled")
